Installation

In [1]:
pip install tensorflow opencv-python numpy scikit-learn 

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/5c/98/d145af334fd5807d6ba1ead447bf0c57a36654ea58e726d70c0d09cae913/tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata
  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/a4/7d/f1c30a92854540bf789e9cd5dde7ef49bbe63f855b85a2e6b3db8135c591/opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/bc/49/d5781eaa1a15acb3b3a3f49dc9e2ff18d92d0ce5c2976f4ab5c0a7360250/numpy-2.3.0-cp312-cp312-win_amd64.whl.metadata
  Using cached numpy-2.3.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/68/c7/4e956281a077f4835458c3f9656c6


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pandas


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/1f/d9/74017c4eec7a28892d8d6e31ae9de3baef71f5a5286e74e6b7aad7f8c837/pandas-2.3.0-cp312-cp312-win_amd64.whl.metadata
  Using cached pandas-2.3.0-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl.metadata
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/5c/23/c7abc0ca0a1526a0774eca151daeb8de62ec457e77262b66b359c3c7679e/tzdata-2025.2-py2.py3-none-any.whl.metadata
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.0-cp312-cp312-win_amd64.whl (11.0 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#!wget -O deploy.prototxt.txt https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
#!wget -O res10_300x300_ssd_iter_140000.caffemodel https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

Importing

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import json
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# from google.colab.patches import cv2_imshow

In [4]:
IMG_SIZE = 160          
THRESHOLD = 0.50  

Embedding Model

In [5]:
def buildEmbeddding():
    baseM = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                             include_top=False, weights='imagenet')
    x = GlobalAveragePooling2D()(baseM.output)
    
    x = Dense(128, activation=None, name='embedding')(x)
    model = Model(inputs=baseM.input, outputs=x)
    model.save("face_embedding_model.keras")
    return model


In [6]:
embeddingModel = buildEmbeddding()
embeddingModel.save("face_model.keras")
#embeddingModel.summary()

Face Detector Model

In [7]:
#Pre trained model
proto ="deploy.prototxt.txt"
caffe ="res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNetFromCaffe(proto, caffe)

In [8]:
def FaceDetection(image):
    h, w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
                                 (104.0, 177.0, 123.0), False, False)
    faceNet.setInput(blob)
    detections = faceNet.forward()
    boxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.3: 
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            x1, y1, x2, y2 = box.astype(int)
            
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)
            boxes.append((x1, y1, x2, y2))
    return boxes

Face Preprocess

In [9]:
def preprocess(img):
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img = preprocess_input(img) # Using  MobileNetV2's input for normalization.
    return img

In [10]:
def prepareEmbeddings(dataset_dir):
    embeddings = []
    labels = []
    for person_name in os.listdir(dataset_dir):
        person_dir = os.path.join(dataset_dir, person_name)
        if not os.path.isdir(person_dir):
            continue
        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            boxes = FaceDetection(img)
            if not boxes:
                print(f"[Warning] No face found in {img_path}")
                continue
            x1, y1, x2, y2 = boxes[0]
            face = img[y1:y2, x1:x2]
            face = preprocess(face)
            emb = embeddingModel.predict(np.expand_dims(face, axis=0))[0]
            emb_norm = emb / np.linalg.norm(emb)
            embeddings.append(emb_norm)
            labels.append(person_name)
    return np.array(embeddings), labels

Face detector in Given Input image

In [11]:
def recognizeFace(image_path, known_embeddings, known_labels):
    image = cv2.imread(image_path)
    if image is None:
        print(f"[Error] Unable to load image at {image_path}")
        return {}, []
    
    image = cv2.resize(image, None, fx=1.5, fy=1.5)
    boxes = FaceDetection(image)
    present = set()
    detection_details = []

    for (x1, y1, x2, y2) in boxes:
        face = image[y1:y2, x1:x2]
        face_proc = preprocess(face)
        emb = embeddingModel.predict(np.expand_dims(face_proc, axis=0))[0]
        emb_norm = emb / np.linalg.norm(emb)

        sims = cosine_similarity([emb_norm], known_embeddings)[0]
        best_idx = np.argmax(sims)
        best_score = sims[best_idx]

        print(f"Comparing face to knowns... Best match: {known_labels[best_idx]} | Score: {best_score:.2f}")

        if best_score > THRESHOLD:
            name = known_labels[best_idx]
        else:
            name = "Unknown"

        detection_details.append({
            "name": name,
            "score": float(best_score),
            "box": (x1, y1, x2, y2)
        })

        if name != "Unknown":
            present.add(name)

        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
        label = f"{name} ({best_score:.2f})"
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    max_width = 1000
    scale = max_width / image.shape[1] if image.shape[1] > max_width else 1.0
    output_img = cv2.resize(image, None, fx=scale, fy=scale)

    cv2.imshow("Face Recognition Result", output_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    all_students = sorted(set(known_labels))
    attendance = {name: ("Present" if name in present else "Absent") 
                  for name in all_students}
    return attendance, detection_details

Saving attendance in CSV 

In [12]:
def save_attendance(attendance_dict):
    with open("attendance_report.json", "w") as jf:
        json.dump(attendance_dict, jf, indent=2)
    
    df = pd.DataFrame(list(attendance_dict.items()), columns=["Name", "Status"])
    df.to_csv("attendance_report.csv", index=False)
    print("[Info] Attendance report saved to attendance_report.json and attendance_report.csv.")

Main fucntion

In [14]:
if __name__ == "__main__":
    
    dataset_folder = "images"      
    test_img = "inputimages/classroom_test2.jpg"           
    print("[Info] Preparing known face embeddings...")
    known_embs, known_labels = prepareEmbeddings(dataset_folder)
    with open("known_faces.pkl", "wb") as f:
        pickle.dump((known_embs, known_labels), f)
    
    print("[Info] Recognizing faces in classroom image...")
    attendance, detection_details = recognizeFace(test_img, known_embs, known_labels)

    print("\n--- Attendance Report ---")
    for name, status in attendance.items():
        print(f"{name}: {status}")
        

    save_attendance(attendance)

[Info] Preparing known face embeddings...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━